In [1]:
import pandas as pd

In [ ]:
#!pip install xlrd

# P4C1T8

In [22]:
df_ill = pd.read_excel('/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/ill_sheets/ILL138_GENEWIZ.xls',
                            sheet_name = 'Individual Library Information', header = 1)
df_samplesheet = pd.read_csv('/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/metadata/P4C1T8-sample_sheets - P4C1T8.tsv', sep = '\t')
df_samplesheet = df_samplesheet.loc[df_samplesheet['sample_type'].str.contains('DNA')].copy()

df_ill = df_ill.dropna(subset=['Sample Name/Pool Name*'])
df_ill['sample_id']= df_ill['Sample Name/Pool Name*'].apply(lambda x: str(x).split('_')[0])
df_ill['project'] = df_ill['Library Name*'].apply(lambda x: str(x).split('_')[1])
#correct library name in ILL138 run
df_ill['ill'] = df_ill['Library Name*'].apply(lambda x:  'ILL138' if str(x).split('_')[0] == 'ILL139' else str(x).split('_')[0])
df_ill['file_name'] = df_ill['Sample Name/Pool Name*'].apply(lambda x: str(x).replace('_', '-'))
df_merge = pd.merge(df_ill[['sample_id', 'ill', 'file_name', 'project']], df_samplesheet, how = 'right', on = ['sample_id', 'project'])
df_merge.head(10)

,sample_id,ill,file_name,project,sample_type,date_collected,time_point_h,cage_id,mouse_id,gender,...,strains,tube_type,tube_weight_empty,tube_weight_full,stool_weight_mg,cfu_g_stool,dna_conc_ng_ul,cfu_incubation_time,cfu_g_stool_spec,cfu_g_stool_cmr
0,s1,ILL138,s1-A12,P4C1T8,gavage DNA,241217,0 h,c1,mA,NaN,...,E. coli|E.coli,2ml_screwcap,1653.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s2,ILL138,s2-B12,P4C1T8,gavage DNA,241217,0 h,c1,mB,NaN,...,E. coli|E.coli,2ml_screwcap,1690.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s3,ILL138,s3-C12,P4C1T8,gavage DNA,241217,0 h,c2,mA,NaN,...,E. coli|E.coli,2ml_screwcap,1644.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,ILL138,s4-D12,P4C1T8,gavage DNA,241217,0 h,c2,mB,NaN,...,E. coli|E.coli,2ml_screwcap,1651.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,s5,ILL138,s5-E12,P4C1T8,gavage DNA,241217,0 h,c3,mA,NaN,...,E. coli|E.coli,2ml_screwcap,1657.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,s6,ILL138,s6-F12,P4C1T8,gavage DNA,241217,0 h,c3,mB,NaN,...,E. coli|E.coli,2ml_screwcap,1697.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,s7,ILL138,s7-G12,P4C1T8,gavage DNA,241217,0 h,c4,mA,NaN,...,E. coli|E.coli,2ml_screwcap,1680.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,s8,ILL138,s8-H12,P4C1T8,gavage DNA,241217,0 h,c4,mB,NaN,...,E. coli|E.coli,2ml_screwcap,1686.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,s9,ILL138,s9-H11,P4C1T8,gavage DNA,241217,0 h,c5,mAB,NaN,...,E. coli|E.coli|B.uniformis,2ml_screwcap,1691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,s19,ILL138,s19-A1,P4C1T8,stool DNA,241217,8 h,c1,mA,female,...,E. coli,2ml_screwcap,1674.1,1752.0,77.9,NaN,4.38,NaN,NaN,NaN


## Create readtables for cages
- Read tables are created that all samples are quantifed by the mouse gavage. Meaning gavage mA quantifies ST1 and ST2 in all samples!
- all cages got uique different gavage solution for every mouse


In [62]:
basepath = '/Volumes/sd/faith/MTCSB/projects/P4-barcoding_strains/r2_data/population-bottlenecks'
exportpath = '/Users/ruprec01/Documents/Faith_lab/Git/population_bottlenecks/input_files/fp_inputs/P4C1T8/'
strains_c1toc4_mAgavage = ['ST1', 'ST2']
strains_c1toc4_mBgavage = [ 'ST5', 'ST6']
strains_c5 = ['ST1', 'ST2', 'ST4']
for cage in df_merge['cage_id'].unique():
    if cage != 'c5':
        df_cage = df_merge[df_merge['cage_id'] == cage].copy()
        
        #creates a readtable per mouse gavage
        df_mA_gavage = df_cage[((df_cage['mouse_id'] == 'mA') & (df_cage['sample_type'] == 'gavage DNA'))]
        df_mB_gavage = df_cage[((df_cage['mouse_id'] == 'mB') & (df_cage['sample_type'] == 'gavage DNA'))]
        for idx, row in df_mA_gavage.iterrows():
            df = pd.read_csv(basepath + '/' + row['ill'] + '/readtables/' + row['file_name'] + "/" + row['file_name'] + '_unique_umi_table.csv')
            df['sample'] = row['sample_id']+'_'+row['sample_type']
            df_reads_mA = df
        for idx, row in df_mB_gavage.iterrows():
            df = pd.read_csv(basepath + '/' + row['ill'] + '/readtables/' + row['file_name'] + "/" + row['file_name'] + '_unique_umi_table.csv')
            df['sample'] = row['sample_id']+'_'+row['sample_type']
            df_reads_mB = df

        #get all samples into dataframe   
        df_samples = df_cage[df_cage['sample_type'] != 'gavage DNA'].copy()
        #print(df_samples)
        df_reads = pd.DataFrame()
        for idx, row in df_samples.iterrows():
            df = pd.read_csv(basepath + '/' + row['ill'] + '/readtables/' + row['file_name'] + "/" + row['file_name'] + '_unique_umi_table.csv')
            df['sample'] = row['sample_id']+'_'+row['sample_type']
            df_reads = pd.concat([df_reads,df])

        #pivot all dataframes
        df_reads_pivot = df_reads.pivot(index = ['strain', 'umi_seq'], columns = 'sample', values = 'counts').fillna(0)
        df_gavage_mA = df_reads_mA.pivot(index = ['strain', 'umi_seq'], columns = 'sample', values = 'counts').fillna(0)
        df_gavage_mB = df_reads_mB.pivot(index = ['strain', 'umi_seq'], columns = 'sample', values = 'counts').fillna(0)

        df_reads_mA = pd.concat([df_gavage_mA, df_reads_pivot], axis = 1).fillna(0)
        df_reads_mB = pd.concat([df_gavage_mB, df_reads_pivot], axis = 1).fillna(0)
        #replace spaces with '-'
        df_reads_mA.columns = df_reads_mA.columns.str.replace(' ', '-')
        df_reads_mB.columns = df_reads_mB.columns.str.replace(' ', '-')
        df_reads_mA.reset_index(inplace = True)
        df_reads_mB.reset_index(inplace = True)
        
        #export csv files
        for strain in strains_c1toc4_mAgavage:
            df_reads_mA.loc[df_reads_mA['strain'] == strain].drop('strain', axis = 1).to_csv(exportpath + '/' + cage + '_' + strain + '_readstable.csv', index = False)
        for strain in strains_c1toc4_mBgavage:
            df_reads_mB.loc[df_reads_mB['strain'] == strain].drop('strain', axis = 1).to_csv(exportpath + '/' + cage + '_' + strain + '_readstable.csv', index = False)        
    else:
        df_cage = df_merge[df_merge['cage_id'] == cage].copy()
        
        #creates a readtable per mouse gavage
        df_mA_gavage = df_cage[df_cage['sample_type'] == 'gavage DNA']
        for idx, row in df_mA_gavage.iterrows():
            df = pd.read_csv(basepath + '/' + row['ill'] + '/readtables/' + row['file_name'] + "/" + row['file_name'] + '_unique_umi_table.csv')
            df['sample'] = row['sample_id']+'_'+row['sample_type']
            df_reads_mA = df
        
        #get all samples into dataframe   
        df_samples = df_cage[df_cage['sample_type'] != 'gavage DNA'].copy()
        #print(df_samples)
        df_reads = pd.DataFrame()
        for idx, row in df_samples.iterrows():
            df = pd.read_csv(basepath + '/' + row['ill'] + '/readtables/' + row['file_name'] + "/" + row['file_name'] + '_unique_umi_table.csv')
            df['sample'] = row['sample_id']+'_'+row['sample_type']
            df_reads = pd.concat([df_reads,df])

        #pivot all dataframes
        df_reads_pivot = df_reads.pivot(index = ['strain', 'umi_seq'], columns = 'sample', values = 'counts').fillna(0)
        df_gavage_mA = df_reads_mA.pivot(index = ['strain', 'umi_seq'], columns = 'sample', values = 'counts').fillna(0)

        df_reads_mA = pd.concat([df_gavage_mA, df_reads_pivot], axis = 1).fillna(0)
        #replace spaces with '-'
        df_reads_mA.columns = df_reads_mA.columns.str.replace(' ', '-')
        df_reads_mA.reset_index(inplace = True)
        
        #export csv files
        for strain in strains_c5:
            df_reads_mA.loc[df_reads_mA['strain'] == strain].drop('strain', axis = 1).to_csv(exportpath + '/' + cage + '_' + strain + '_readstable.csv', index = False)

```cmd
rsync -avzP \
/Volumes/sd/faith/MTCSB/projects/P4-barcoding_strains/r2_data/population-bottlenecks/fp_inputs/P4C1T8/ \
ruprec02@minerva.hpc.mssm.edu:/sc/arion/work/ruprec02/population-bottlenecks/experiments/P4C1T8/fp_inputs
```

# P4C2T4 & T5
- both experiments share the same gavage solution and should be possible to combine readtables